In [1]:
import gensim
import gensim.downloader

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchtext import data
from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator
from torchtext.vocab import Vectors
import time
import pandas as pd
import math
import heapq
import random
from tqdm import tqdm
from torchtext.vocab import GloVe
from gensim.models.word2vec import Word2Vec
from gensim.corpora.dictionary import Dictionary
from nltk import word_tokenize

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(1)

In [4]:
tokenize = word_tokenize
Article = data.Field(sequential=True, tokenize=tokenize, lower=True, init_token='<bos>', eos_token='<eos>',
                  pad_token='<pad>', unk_token='<oov>')
Title = data.Field(sequential=True, tokenize=tokenize, lower=True, eos_token='<eos>', 
                  pad_token='<pad>', unk_token='<oov>')
Len = data.Field(sequential=False, use_vocab=False)
ID = data.Field(sequential=False, use_vocab=False)

In [5]:
pmi_dct = dict()

In [6]:
train_path = 'train_data.csv'
valid_path = 'valid_data.csv'
test_path = 'test_data.csv'


class MyDataset(data.Dataset):
    def __init__(self, csv_path, article_field, title_field, cat, **kwargs):

        csv_data = pd.read_csv(csv_path)

        fields = [("id", ID), ("article", article_field), ("title", title_field), ("art_len", Len), ("title_len", Len)]
        examples = []
    
        
        for id, text, label in tqdm(zip(csv_data.index, csv_data['article'], csv_data['title'])):
            examples.append(data.Example.fromlist([self.getID(id, cat), text, label, len(word_tokenize(text))+2, len(word_tokenize(label))+1], fields))
        super(MyDataset, self).__init__(examples, fields)

    def getID(self, id, cat):
        if cat == 'train':
            return id
        elif cat == 'valid':
            return id + 100000
        elif cat == 'test':
            return id + 110000
        
    def shuffle(self, text):
        text = np.random.permutation(text.strip().split())
        return ' '.join(text)

In [7]:
train = MyDataset(train_path, article_field=Article, title_field=Title, cat='train')
valid = MyDataset(valid_path, article_field=Article, title_field=Title, cat='valid')
test = MyDataset(test_path, article_field=Article, title_field=Title, cat='test')

100000it [01:28, 1125.36it/s]
10000it [00:09, 1106.46it/s]
10000it [00:09, 1095.92it/s]


In [8]:
from collections import defaultdict
vectors = Vectors(name='glove-wiki-gigaword-300.txt')
Article.build_vocab(train, valid, test, vectors=vectors)
default = Article.vocab.stoi['<oov>']
Article.vocab.stoi.default_factory = lambda: default
Title.vocab = Article.vocab

In [9]:
from torchtext.data import Iterator, BucketIterator

train_iter, val_iter = BucketIterator.splits(
        (train, valid), 
        batch_size=16, 
        device=device, 
        sort_key=lambda x: len(x.article),
        sort_within_batch=True,
        repeat=False 
)


# GCN

## layer

In [10]:
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, device, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features)).to(device)
        self.device = device
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features)).to(device)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.mm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [11]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import math
import numpy as np



class GCN(nn.Module):
    def __init__(self, input_dim, nfeat, nhid, nout, dropout, device):
        super(GCN, self).__init__()
        self.input_dim = input_dim
        self.nfeat = nfeat
        self.gc1 = GraphConvolution(nfeat, nhid, device)
        self.gc2 = GraphConvolution(nhid, nout, device)
        self.device = device
        self.dropout = dropout

    def forward(self, input_seq, seq_embed, input_length, id=None):
        pad_len = len(input_seq)
        input_seq = input_seq[:input_length]
        
        adj, uni_words = self.load_data(input_seq, id)
        uni2idx = [list(input_seq).index(ele) for ele in uni_words]
        seq2idx = [list(uni_words).index(ele) for ele in input_seq]
        x = seq_embed[uni2idx]
        x = torch.cat((x, torch.zeros((1, self.nfeat), device=self.device)),0)
        # x: [node_num, feature_dim]
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        x = torch.tanh(x) # [node_num, embded_dim]
        return x[-1,:]

        

    def unranked_unique(self, nparray):
        n_unique = len(np.unique(nparray))
        ranked_unique = np.zeros([n_unique])
        i = 0
        for x in nparray:
            if x not in ranked_unique:
                ranked_unique[i] = x
                i += 1
        return ranked_unique

    def load_data(self, word_seq, id):
        if id is not None:
            id = int(id)
        word_seq = torch.tensor(word_seq, device=self.device)
        words = torch.unique(word_seq, sorted=False)
        word_num = len(words)
        if id is None:
            adj = self.get_pmi(word_seq, words, 3, self.device)
        elif id in pmi_dct.keys():
            adj = pmi_dct[id]
        else:
            adj = self.get_pmi(word_seq, words, 3, self.device)
            adj = torch.tensor(self.normalize(adj), dtype=torch.float32, device=self.device)
            pmi_dct[id] = adj
        words = torch.tensor(words, dtype=torch.int64, device=self.device)
        return adj, words

    def get_pmi(self, word_seq, uni_words, window_size, device):
        word_len = len(word_seq)
        word_num = len(uni_words)
        seq2idx = {int(ele): list(uni_words).index(ele) for ele in uni_words}
        window_num = word_len - window_size + 1
        if window_size > word_len:
            window_size = word_len
        win_num_matrix = torch.zeros((word_num+1, word_num+1), device=device)

        for i in range(window_num):
            words = list(torch.unique(word_seq[i:i+window_size], sorted=False))
            indicies = [seq2idx[int(ele)] for ele in words]
            for ele in indicies:
                win_num_matrix[ele,indicies] += 1

        pos_matrix = win_num_matrix / window_num
        pos_matrix[-1,-1]=1
        diag = torch.mul(pos_matrix, torch.eye(word_num+1, device=device)).inverse()
        pos_matrix = torch.mm(diag, pos_matrix)
        pos_matrix = torch.mm(pos_matrix, diag)
        pmi_matrix = torch.clamp(torch.log2(pos_matrix), min=0)
        diag = torch.diag(pmi_matrix)
        diag = 1 - diag
        pmi_matrix = pmi_matrix + torch.diag_embed(diag)
        pmi_matrix[:,-1] += 1
        pmi_matrix[-1,:] += 1
        pmi_matrix[-1,-1] = 0
        return pmi_matrix

    def normalize(self, mx):
        """Symmetrically normalize adjacency matrix"""
        rowsum = mx.sum(1)
        r_inv = torch.pow(rowsum, -0.5).flatten()
        r_inv = torch.clamp(r_inv, min=0)
        r_mat_inv = torch.diag(r_inv)
        mx = torch.mm(r_mat_inv, mx)
        mx = torch.mm(mx, r_mat_inv)
        return mx


## GAT

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nout, dropout, alpha, nheads, device):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout
        self.device = device
        self.nfeat = nfeat
        self.nhid = nhid
        self.nout = nout
        self.nheads = nheads
        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, device=device, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = GraphAttentionLayer(nhid * nheads, nout, dropout=dropout, alpha=alpha, device=device, concat=False)

#     def forward(self, x, adj):
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
#         x = F.dropout(x, self.dropout, training=self.training)
#         x = F.elu(self.out_att(x, adj))
#         return F.log_softmax(x, dim=1)

    def forward(self, input_seq, seq_embed, input_length, id=None):
        pad_len = len(input_seq)
        input_seq = input_seq[:input_length]
        
        adj, uni_words = self.load_data(input_seq, id)
        uni2idx = [list(input_seq).index(ele) for ele in uni_words]
        seq2idx = [list(uni_words).index(ele) for ele in input_seq]
        x = seq_embed[uni2idx]
        x = torch.cat((x, torch.zeros((1, self.nfeat), device=self.device)),0)
        # x: [node_num, feature_dim]
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.elu(self.out_att(x, adj))
        x = torch.tanh(x) # [node_num, embded_dim]
        return x[-1,:]

    def unranked_unique(self, nparray):
        n_unique = len(np.unique(nparray))
        ranked_unique = np.zeros([n_unique])
        i = 0
        for x in nparray:
            if x not in ranked_unique:
                ranked_unique[i] = x
                i += 1
        return ranked_unique

    def load_data(self, word_seq, id):
        if id is not None:
            id = int(id)
        word_seq = torch.tensor(word_seq, device=self.device)
        words = torch.unique(word_seq, sorted=False)
        word_num = len(words)
        if id is None:
            adj = self.get_pmi(word_seq, words, 3, self.device)
        elif id in pmi_dct.keys():
            adj = pmi_dct[id]
        else:
            adj = self.get_pmi(word_seq, words, 3, self.device)
            adj = torch.tensor(self.normalize(adj), dtype=torch.float32, device=self.device)
            pmi_dct[id] = adj
        words = torch.tensor(words, dtype=torch.int64, device=self.device)
        return adj, words

    def get_pmi(self, word_seq, uni_words, window_size, device):
        word_len = len(word_seq)
        word_num = len(uni_words)
        seq2idx = {int(ele): list(uni_words).index(ele) for ele in uni_words}
        window_num = word_len - window_size + 1
        if window_size > word_len:
            window_size = word_len
        win_num_matrix = torch.zeros((word_num+1, word_num+1), device=device)

        for i in range(window_num):
            words = list(torch.unique(word_seq[i:i+window_size], sorted=False))
            indicies = [seq2idx[int(ele)] for ele in words]
            for ele in indicies:
                win_num_matrix[ele,indicies] += 1

        pos_matrix = win_num_matrix / window_num
        pos_matrix[-1,-1]=1
        diag = torch.mul(pos_matrix, torch.eye(word_num+1, device=device)).inverse()
        pos_matrix = torch.mm(diag, pos_matrix)
        pos_matrix = torch.mm(pos_matrix, diag)
        pmi_matrix = torch.clamp(torch.log2(pos_matrix), min=0)
        diag = torch.diag(pmi_matrix)
        diag = 1 - diag
        pmi_matrix = pmi_matrix + torch.diag_embed(diag)
#         pmi_matrix[:,-1] += 1
        pmi_matrix[-1,:] += 1
        pmi_matrix[-1,-1] = 0
        return pmi_matrix

    def normalize(self, mx):
        """Symmetrically normalize adjacency matrix"""
        rowsum = mx.sum(1)
        r_inv = torch.pow(rowsum, -0.5).flatten()
        r_inv = torch.clamp(r_inv, min=0)
        r_mat_inv = torch.diag(r_inv)
        mx = torch.mm(r_mat_inv, mx)
        mx = torch.mm(mx, r_mat_inv)
        return mx
    


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, device, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.device =device
        self.out_features = out_features
        self.alpha = alpha         # 学习因子
        self.concat = concat
        # 建立就是0的矩阵，大小为(输入维度，输出维度)
        self.W = nn.Parameter(torch.empty(size=(in_features, out_features))).to(device)
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        # self.a.shape [16, 1], 因为GAT的注意力机制打分用的是拼接方法，所以这里的a维度是[2*out_features, 1]
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1))).to(device)
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        # adj: 邻接矩阵， 维度： N*N
        # h: [N, in_features]
        
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        a_input = self._prepare_attentional_mechanism_input(Wh)  # a_input: [N, N, 2*out_features]
        # self.a: [2*out_features, 1]
        # e: [N, N]
        e = self.leakyrelu(torch.matmul(a_input, self.a).squeeze(2))
        # 这里采用了Local Attention 机制，即只对每个节点的一节邻节点进行聚合。所以对于一个邻接矩阵，只对值为1的位置赋值。
        zero_vec = -9e15*torch.ones_like(e)
        # attention 是一个N*N的矩阵，
        attention = torch.where(adj > 0, adj, zero_vec)

        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh: [N, out_features]

        N = Wh.size()[0] # number of nodes

        # Below, two matrices are created that contain embeddings in their rows in different orders.
        # (e stands for embedding)
        # These are the rows of the first matrix (Wh_repeated_in_chunks): 
        # e1, e1, ..., e1,            e2, e2, ..., e2,            ..., eN, eN, ..., eN
        # '-------------' -> N times  '-------------' -> N times       '-------------' -> N times
        # 
        # These are the rows of the second matrix (Wh_repeated_alternating): 
        # e1, e2, ..., eN, e1, e2, ..., eN, ..., e1, e2, ..., eN 
        # '----------------------------------------------------' -> N times
        # 
        
        Wh_repeated_in_chunks = Wh.repeat_interleave(N, dim=0)  # Wh_repeated_in_chunks: []
        Wh_repeated_alternating = Wh.repeat(N, 1)
        # Wh_repeated_in_chunks.shape == Wh_repeated_alternating.shape == (N * N, out_features)

        # The all_combination_matrix, created below, will look like this (|| denotes concatenation):
        # e1 || e1
        # e1 || e2
        # e1 || e3
        # ...
        # e1 || eN
        # e2 || e1
        # e2 || e2
        # e2 || e3
        # ...
        # e2 || eN
        # ...
        # eN || e1
        # eN || e2
        # eN || e3
        # ...
        # eN || eN

        all_combinations_matrix = torch.cat([Wh_repeated_in_chunks, Wh_repeated_alternating], dim=1)
        # all_combinations_matrix.shape == (N * N, 2 * out_features)

        return all_combinations_matrix.view(N, N, 2 * self.out_features)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'

## Encoder

In [14]:
class Encoder(nn.Module):
    def __init__(self, gnn, input_dim, emb_dim, hid_dim, n_layers, device, dropout=0.5, bidirectional=True):
        super(Encoder, self).__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.GNN = gnn
        self.embedding = nn.Embedding(input_dim, emb_dim)
        weight_matrix = Article.vocab.vectors
        self.embedding.weight.data.copy_(weight_matrix)
        self.device = device
        self.gru = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout, bidirectional=bidirectional)
        
#     def forward(self, input_seqs, input_lengths, ids, hidden):
#         # input_seqs = [seq_len, batch]
#         embedded = self.embedding(input_seqs)
#         # embedded = [pad_len, batch, embed_dim]
        
#         gcn_outputs = torch.zeros((embedded.shape[1], self.hid_dim), device=self.device)
#         for i in range(embedded.shape[1]):
#             gcn_input = embedded[:,i,:]
#             if ids is not None:
#                 gcn_output = self.GCN(input_seqs[:,i], gcn_input, input_lengths[i], ids[i]) # pad_len, emb_dim
#             else:
#                 gcn_output = self.GCN(input_seqs[:,i], gcn_input, input_lengths[i])
#             gcn_outputs[i,:] = gcn_output
#         packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
#         # gcn_outputs [batch, embed_dim]
#         outputs, hidden = self.gru(packed, hidden)        
#         gcn_outputs = gcn_outputs.repeat(hidden.shape[0],1,1)
#         hidden = torch.cat((hidden, gcn_outputs), -1)
#         outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
#         # outputs = [seq_len, batch, hid_dim * n directions]
#          # hidden = [n_layers * n_directions, batch, hid_dim]
#         # output_lengths = [batch]
#         return outputs, hidden
    def forward(self, input_seqs, input_lengths, ids, hidden):
        # input_seqs = [seq_len, batch]
        embedded = self.embedding(input_seqs)
        # embedded = [pad_len, batch, embed_dim]
        
        gnn_outputs = torch.zeros((embedded.shape[1], self.hid_dim), device=self.device)
        for i in range(embedded.shape[1]):
            gnn_input = embedded[:,i,:]
            if ids is not None:
                gnn_output = self.GNN(input_seqs[:,i], gnn_input, input_lengths[i], ids[i]) # pad_len, emb_dim
            else:
                gnn_output = self.GNN(input_seqs[:,i], gnn_input, input_lengths[i])
            gnn_outputs[i,:] = gnn_output
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, input_lengths, enforce_sorted=False)
        # gcn_outputs [batch, embed_dim]
        outputs, hidden = self.gru(packed, hidden)        
        gnn_outputs = gnn_outputs.repeat(hidden.shape[0],1,1)
        hidden = torch.cat((hidden, gnn_outputs), -1)
        outputs, output_lengths = torch.nn.utils.rnn.pad_packed_sequence(outputs)
        # outputs = [seq_len, batch, hid_dim * n directions]
         # hidden = [n_layers * n_directions, batch, hid_dim]
        # output_lengths = [batch]
        return outputs, hidden

In [15]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout=0.5, bidirectional=True):
        super(Decoder, self).__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        weight_matrix = Article.vocab.vectors
        self.embedding.weight.data.copy_(weight_matrix)
        self.gru = nn.GRU(emb_dim, hid_dim*2, n_layers, dropout=dropout, bidirectional=bidirectional)
        
        if bidirectional:
            self.fc_out = nn.Linear(hid_dim*4, output_dim)
        else:
            self.fc_out = nn.Linear(hid_dim*2, output_dim)
        self.dropout = nn.Dropout(dropout)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, token_inputs, hidden):
        # token_inputs = [batch]
        batch_size = token_inputs.size(0)
        embedded = self.dropout(self.embedding(token_inputs).view(1, batch_size, -1))
        # embedded = [1, batch, emb_dim]
        output, hidden = self.gru(embedded, hidden)
        # output = [1, batch,  n_directions * hid_dim]
        # hidden = [n_layers * n_directions, batch, hid_dim]
        
        output = self.fc_out(output.squeeze(0))
        output = self.softmax(output)
        # output = [batch, output_dim]
        return output, hidden

In [16]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 device, 
                 predict=False, 
                 basic_dict=None,
                 max_len=100
                 ):
        super(Seq2Seq, self).__init__()
        
        self.device = device

        self.encoder = encoder
        self.decoder = decoder

        self.predict = predict  
        self.basic_dict = basic_dict  
        self.max_len = max_len  

        self.enc_n_layers = self.encoder.gru.num_layers
        self.enc_n_directions = 2 if self.encoder.gru.bidirectional else 1
        self.dec_n_directions = 2 if self.decoder.gru.bidirectional else 1

        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        assert self.enc_n_directions >= self.dec_n_directions, \
            "If decoder is bidirectional, encoder must be bidirectional either!"
        
    def forward(self, input_batches, input_lengths, target_batches=None, target_lengths=None, ids=None, teacher_forcing_ratio=0.5):
        # input_batches = target_batches = [seq_len, batch]
        batch_size = input_batches.size(1)
        BOS_token = self.basic_dict["<bos>"]
        EOS_token = self.basic_dict["<eos>"]
        PAD_token = self.basic_dict["<pad>"]
        encoder_hidden = torch.zeros(self.enc_n_layers*self.enc_n_directions, batch_size, self.encoder.hid_dim, device=self.device)
        
        # encoder_output = [seq_len, batch, hid_dim * n directions]
        # encoder_hidden = [n_layers*n_directions, batch, hid_dim*2]
        encoder_output, encoder_hidden = self.encoder(
            input_batches, input_lengths, ids, encoder_hidden)

        decoder_input = torch.tensor([BOS_token] * batch_size, dtype=torch.long, device=self.device)
        if self.enc_n_directions == self.dec_n_directions:
            decoder_hidden = encoder_hidden
        else:
            L = encoder_hidden.size(0)
            decoder_hidden = encoder_hidden[range(0, L, 2)] + encoder_hidden[range(1, L, 2)]

        if self.predict:
            assert batch_size == 1, "batch_size of predict phase must be 1!"
            output_tokens = []

#             return beam_search(3, decoder_inpuy, decoder_hidden)
            prev_beam = Beam()
            prev_beam.add(0, False, [decoder_input], decoder_input, decoder_hidden)
            while True:
                cur_beam = Beam()
                for prob, complete, seq_list, decoder_input, decoder_hidden in prev_beam:
                    if complete:
                        cur_beam.add(prob, complete, seq_list, decoder_input, decoder_hidden)
                    else:
                        # decoder_output_t: [batch_size, vocab_dim]
                        # decoder_hidden: [num_layer*num_direction, batch_size, hidden_dim]
                        decoder_output_t, decoder_hidden = self.decoder(decoder_input, decoder_hidden)
                        # value: [batch_size, beam_width] log_softmax, 
                        # index: [batch_size, beam_width]
                        value, index = torch.topk(decoder_output_t, 3)
                        seq_len = 0
                        for m, n in zip(value[0], index[0]):
                            seq_len += 1
                            cur_prob = (prob+m.item())/ seq_len
                            decoder_input = torch.LongTensor([n.item()]).to(self.device)
                            cur_seq_list = seq_list + [decoder_input]
                            if int(n) == EOS_token or len(cur_seq_list) == self.max_len:
                                cur_complete = True
                            else:
                                cur_complete = False
                            cur_beam.add(cur_prob, cur_complete, cur_seq_list, decoder_input, decoder_hidden)
                    
                best_prob, best_complete, best_seq, _, _ = max(cur_beam)
                if best_complete or len(best_seq) >= self.max_len: 
                    best_seq = [i.item() for i in best_seq]
                    return best_seq
                else:
                    prev_beam = cur_beam

        else:
            max_target_length = max(target_lengths)
            all_decoder_outputs = torch.zeros((max_target_length, batch_size, self.decoder.output_dim), device=self.device)

            for t in range(max_target_length):
                use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = target_batches[t]  
                else:
                    decoder_output, decoder_hidden = self.decoder(
                        decoder_input, decoder_hidden
                    )
                    # [batch, 1]
                    topv, topi = decoder_output.topk(1)
                    all_decoder_outputs[t] = decoder_output
                    decoder_input = topi.squeeze(1)  
     
            loss_fn = nn.NLLLoss(ignore_index=PAD_token)
            loss = loss_fn(
                all_decoder_outputs.reshape(-1,self.decoder.output_dim ),  # [batch*seq_len, output_dim]
                target_batches.reshape(-1)                                                 # [batch*seq_len]
            )
            return loss

In [17]:
class Beam:
    """保存每一个时间步的数据"""
 
    def __init__(self, beam_width=1):
        self.heapq = list()
        self.beam_width = beam_width
 
    def add(self, prob, complete, seq_list, decoder_input, decoder_hidden):
        heapq.heappush(self.heapq, [prob, complete, seq_list, decoder_input, decoder_hidden])
        # 保证最终只有一个beam width个结果
        if len(self.heapq) > self.beam_width:
            heapq.heappop(self.heapq)
 
    def __iter__(self):
        for item in self.heapq:
            yield item

In [18]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [19]:
def train(
    model,
    data_loader, 
    optimizer, 
    clip=1, 
    teacher_forcing_ratio=0.5, 
    print_every=1  
    ):
    model.predict = False
    model.train()

    if print_every == 0:
        print_every = 1

    print_loss_total = 0  
    start = time.time()
    epoch_loss = 0
    step = 0
    for batch in tqdm(data_loader, position=0, leave=True):
        step += 1
        ids = batch.id
        input_batchs = batch.article
        target_batchs = batch.title
        input_lens = list(batch.art_len)
        target_lens = list(batch.title_len)
        
        optimizer.zero_grad()
        loss = model(input_batchs, input_lens, target_batchs, target_lens, ids, teacher_forcing_ratio)
        print_loss_total += loss.item()
        epoch_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        if step % 500 == 0:
            print_loss_avg = print_loss_total / 500
            print_loss_total = 0
            print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [20]:
def evaluate(
    model,
    data_loader, 
    print_every=None
    ):
    model.predict = False
    model.eval()
    if print_every == 0:
        print_every = 1

    print_loss_total = 0  
    start = time.time()
    epoch_loss = 0
    with torch.no_grad():
        for i, batch in enumerate(data_loader):
            ids = batch.id
            input_batchs = batch.article
            target_batchs = batch.title
            input_lens = list(batch.art_len)
            target_lens = list(batch.title_len)
        

            loss = model(input_batchs, input_lens, target_batchs, target_lens, ids, teacher_forcing_ratio=0)
            print_loss_total += loss.item()
            epoch_loss += loss.item()

            if print_every and (i+1) % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('\tCurrent Loss: %.4f' % print_loss_avg)

    return epoch_loss / len(data_loader)

In [21]:
def summary(
    model,
    sample, 
    idx2token=None
    ):
    model.predict = True
    model.eval()

    # shape = [seq_len, 1]
    input_batch = sample['src']
    input_len = sample['src_len']

    output_tokens = model(input_batch, input_len)
    output_tokens = [idx2token[t] for t in output_tokens]

    return " ".join(output_tokens)


In [22]:
INPUT_DIM = len(Article.vocab.stoi)
OUTPUT_DIM = len(Article.vocab.stoi)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = 256
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5
LEARNING_RATE = 1e-4
N_EPOCHS = 100
CLIP = 1

bidirectional = True
gcn = GCN(INPUT_DIM, ENC_EMB_DIM, HID_DIM,HID_DIM,0.5, device).to(device)
gat = GAT(ENC_EMB_DIM, HID_DIM,HID_DIM,0.5, 0.05, 4, device).to(device)
enc = Encoder(gcn, INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, device, ENC_DROPOUT, bidirectional)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT, bidirectional)
model = Seq2Seq(enc, dec, device, basic_dict=Article.vocab.stoi).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

In [23]:
best_valid_loss = float('inf')
count = 0

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    train_loss = train(model, train_iter, optimizer, CLIP)
    valid_loss = evaluate(model, val_iter)
    end_time = time.time()
    if count == 5:
        break
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'gcn+seq2seq(graph).pt')
    else:
        count += 1

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
        
    print(f'\tTrain Loss: {train_loss:.3f} | Val. Loss: {valid_loss:.3f}')

  0%|                                                                                         | 0/6250 [00:00<?, ?it/s]D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  8%|██████▏                                                                    

	Current Loss: 7.6603


 16%|███████████▊                                                              | 1000/6250 [1:42:36<8:39:58,  5.94s/it]

	Current Loss: 7.0288


 24%|█████████████████▊                                                        | 1500/6250 [2:34:00<7:46:11,  5.89s/it]

...

	Current Loss: 3.3862


 56%|█████████████████████████████████████████▍                                | 3500/6250 [3:17:12<2:12:58,  2.90s/it]

	Current Loss: 3.3555


 58%|███████████████████████████████████████████▏                              | 3648/6250 [3:25:38<2:26:40,  3.38s/it]


KeyboardInterrupt: 

In [24]:
from rouge import Rouge

model.load_state_dict(torch.load('gcn+seq2seq(graph).pt'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [25]:
dct = Article.vocab.stoi
r1 = 0
r2 = 0
rl = 0
rouge = Rouge()
for i in range(10000):
    sample = test[i].article
    label = ' '.join(test[i].title)
    sample = [dct['<bos>']] + [dct[ele] for ele in sample] + [dct['<eos>']]
    test_sample = {}
    test_sample["src"] = torch.tensor(sample, dtype=torch.long, device=device).reshape(-1, 1)
    test_sample["src_len"] = [len(sample)]
    predict = ''.join(summary(model, test_sample, Article.vocab.itos))[6:-6]
    score = rouge.get_scores(label, predict)[0]
    r1 += score['rouge-1']['r']
    r2 += score['rouge-2']['r']
    rl += score['rouge-l']['r']
    print(r1/(i+1),r2/(i+1),rl/(i+1))
    print(i)
    
print(r1/10000)
print(r2/10000)
print(rl/10000)

D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


0.04 0.0 0.07142857142857142
0
0.10333333333333333 0.0 0.15336134453781514
1
0.10222222222222221 0.0 0.13927793339558045
2
0.07666666666666666 0.0 0.10445845004668534
3
0.07671794871794871 0.0 0.09895137542196367
4
0.06393162393162392 0.0 0.08245947951830306
5
0.0698361287834972 0.0 0.08853669672997404
6
0.07896375554270291 0.0 0.10246960963872728
7
0.11059404533088743 0.011111111111111112 0.11330631967886869
8
0.12175686302002091 0.01 0.1292484149837091
9
0.1272169829107628 0.009090909090909092 0.15386219543973556
10
0.11661556766819925 0.008333333333333333 0.14104034581975758
11
0.10764513938603007 0.007692307692307693 0.130191088449007
12
0.11016028249110955 0.0071428571428571435 0.13279648689312556
13
0.10281626365836892 0.006666666666666667 0.12394338776691718
14
0.10420274717972086 0.00625 0.13008581492037374
15
0.09807317381620786 0.0058823529411764705 0.12243370816035176
16
0.13892096045604818 0.03888888888888889 0.16192813178107296
17
0.14213564674783513 0.03684210526315789 0.

0.20891140600789718 0.05048983580384547 0.23103663584055734
134
0.20737529272842736 0.05011858701117014 0.22933783704761204
135
0.20732145847493516 0.049752757908898826 0.22948865575529373
136
0.20726840442801536 0.04939223067767492 0.22943600043983667
137
0.20667654540335337 0.0490368908886269 0.2289844224990225
138
0.20520028436475798 0.048686627382279565 0.22734881948117236
139
0.20492699629597722 0.04834133215261801 0.2269184496030553
140
0.20781754290927748 0.04858775469614417 0.2279612774227521
141
0.20776287477704478 0.048247980187779524 0.22776574401420138
142
0.2063200770355375 0.04791292476980883 0.2261840374585472
143
0.20575924891805106 0.04758249080587912 0.22548621651055722
144
0.20532841257516812 0.04725658333460597 0.22508334288148948
145
0.20490343795123597 0.04693510997858824 0.22452398097656198
146
0.20389433063775164 0.046617980857111295 0.22375767780179542
147
0.2029207010993222 0.046305108502365584 0.22281523253690644
148
0.20212345198088227 0.04599640777901648 0.

0.19945039087925212 0.0472775207335484 0.21688068751905684
263
0.19995560953002225 0.04709911499493123 0.21747736416992833
264
0.19974095364027458 0.04692205065284503 0.21728634651014161
265
0.20024129963163437 0.04674631263541864 0.21754263092663653
266
0.20173293657330738 0.04843755773752528 0.2189697106619849
267
0.20191236803586016 0.048257492467125566 0.2192178317588336
268
0.2022227455087432 0.048078761013543615 0.21964048176466508
269
0.20174011018003407 0.04790134861127962 0.21911384927447475
270
0.20099841859848983 0.04772524071197344 0.2183082836521421
271
0.2013610617538067 0.047550422980427755 0.2187296208304615
272
0.2012897373744796 0.0477418447943678 0.2187423675508694
273
0.20116383529917845 0.04756823808602464 0.2186080977845687
274
0.20109374235178354 0.04739588939730716 0.21847480098758904
275
0.20135234715458314 0.04722478510345406 0.2185886103703053
276
0.2014274186476322 0.047689698064359964 0.21915124126825386
277
0.20150195199377768 0.04796679592076011 0.2191622

0.19574031804625971 0.042650520014784664 0.21612420975157984
392
0.1958780329750763 0.04254226996398572 0.2160832853613474
393
0.19633150630931662 0.042796231667512695 0.21648560615790097
394
0.19611630329113933 0.042688160375423015 0.21625458189992647
395
0.1958322152056033 0.04258063352309198 0.21596174919992664
396
0.1955914307452877 0.042473647006702295 0.21583789220863706
397
0.19551893760223352 0.04236719676357773 0.21601302101441563
398
0.19523847359156124 0.042261278771668784 0.2157507662396574
399
0.19520500652978226 0.042405265607649664 0.2156661463283819
400
0.19534131248368827 0.04265514589931507 0.21575155392457995
401
0.1962100798111594 0.043045579780458205 0.21676705875355123
402
0.19572441129677534 0.042939031315655095 0.21623050662792362
403
0.19548805472567218 0.042833009016110264 0.21607647118895631
404
0.19519602276064116 0.042727508993903096 0.21574951764743996
405
0.19502355096024646 0.042786327563123644 0.21566614827193814
406
0.19473408901407693 0.04268145911321

...

0.19355622411121354 0.037065932819279994 0.21292181020193485
9429
0.19353570070710888 0.037062002596311136 0.21289923340093794
9430
0.19353638606538845 0.03705807320672289 0.21291200207141422
9431
0.1935291204673745 0.03705414465025022 0.21290709956580575
9432
0.1935557173853284 0.03705021692662819 0.21293753129152487
9433
0.193535202735897 0.03704629003559198 0.2129149623957865
9434
0.1935382429032864 0.037042363976876894 0.21292772398660226
9435
0.19354952421695562 0.037038438750218325 0.21294754726476411
9436
0.1935555054074391 0.03703451435535181 0.21296030269876162
9437
0.19356678250189746 0.037030590792012964 0.21296128393824923
9438
0.1935595190715477 0.03702666805993754 0.21295637984743657
9439
0.19353901705702894 0.037022746158861385 0.21293382329835836
9440
0.19351851938523726 0.037018825088520474 0.21291127152719774
9441
0.19355097532938792 0.03701490484865089 0.21295932356523012
9442
0.19358342440019166 0.037056365725792226 0.2130073654270579
9443
0.1935746925512463 0.03705

0.19356929004394263 0.03716232058804777 0.21294360314989966
9556
0.19354903797342118 0.03715843250261274 0.2129213240535249
9557
0.19354041385721005 0.03715454523066979 0.2129106733355688
9558
0.1935332443578526 0.037150658771963656 0.2129058360963775
9559
0.19352794406483925 0.03714677312623915 0.21290099986905509
9560
0.1935077047902037 0.03714288829324122 0.21287873454800624
9561
0.1935048980310148 0.037139004272714896 0.21287738782265353
9562
0.19350477285266648 0.037143485765367266 0.2128700665925233
9563
0.1934845423484477 0.037139602494508366 0.21284781148885445
9564
0.19346431607389739 0.03713572003553968 0.21282556103814476
9565
0.19346253978793118 0.0371318383882066 0.2128274366177132
9566
0.19344232004087977 0.037127957552254655 0.21280519294749814
9567
0.19344532765945865 0.03712407752742947 0.2127978830875242
9568
0.1934296571195642 0.03712019831347675 0.21278217797957358
9569
0.1934225074322672 0.03711631991014236 0.212773006296575
9570
0.19342188869977323 0.0371124423171

0.1935906162924265 0.03716148415810362 0.21293097126575808
9683
0.19358277493388554 0.037157647143735206 0.21291931086604038
9684
0.1935885995493167 0.03715381092164727 0.2129268263497125
9685
0.19358151907037074 0.037149975491594454 0.2129195929767908
9686
0.1935797528718948 0.03714614085333149 0.2129163826742352
9687
0.1935735348494427 0.037142307006613216 0.21291028604622117
9688
0.19358304369886115 0.03713847395119458 0.2129055137428796
9689
0.19357780936795188 0.03713464168683061 0.21289828555477874
9690
0.1936020555110806 0.03714800652638693 0.21292790810063564
9691
0.19360271556931735 0.03714417406930178 0.2129206790935952
9692
0.19360654954035098 0.03714893878554523 0.21292450406996266
9693
0.19358657980857785 0.03714510702290618 0.21290254176938814
9694
0.19356661419597385 0.03714127605064722 0.21288058399899112
9695
0.19356251803630076 0.03713744586852382 0.21287581820365933
9696
0.19356318183110008 0.03713361647629155 0.21288332887261277
9697
0.1935432248064758 0.03712978787

0.19362192418502613 0.037323153154619974 0.21289832327459243
9810
0.19361145610173264 0.03731934932735187 0.21288794952691986
9811
0.19360191656681958 0.03731554627534664 0.21287644560869637
9812
0.1936113023798568 0.037311743998367286 0.21288386743874585
9813
0.19362978314375084 0.03730794249617693 0.21292330871562423
9814
0.19363916420554492 0.037304141768538766 0.21294236705825711
9815
0.1936286996977221 0.03730034181521611 0.2129319941076666
9816
0.19362029497287114 0.03729654263597235 0.2129216232701237
9817
0.19363191245048797 0.03730123117764639 0.2129406759615108
9818
0.19364352756201972 0.03731440484724812 0.21296425068335223
9819
0.1936398875263085 0.03731626220909603 0.212968021760566
9820
0.19370501616057922 0.037393912762729804 0.21303118255384365
9821
0.19372601738055678 0.03739010599160461 0.2130298559547849
9822
0.19374588330803116 0.03740426320681672 0.2130473219084352
9823
0.19373541645893202 0.037400456157126456 0.2130348905363233
9824
0.19371569984826043 0.037396649

0.1934038916573231 0.037256446132405655 0.2126257228798235
9937
0.1934131793516643 0.03726043714063493 0.2126244525585759
9938
0.19340090725831188 0.03725668860571132 0.2126131221307531
9939
0.1933814523838266 0.03725294082494422 0.21259173463229913
9940
0.1933620014230155 0.03724919379810607 0.21257035143629913
9941
0.19335512603315097 0.037245447524969374 0.21256334015111567
9942
0.19333568163190065 0.03724170200530677 0.21254196411127746
9943
0.19333635174938363 0.03723795723889095 0.21255411005086744
9944
0.193342048878707 0.03724479669200335 0.2125693003035907
9945
0.1933226116565417 0.03724105236741382 0.2125479301115425
9946
0.19333668591485256 0.03723730879560366 0.2125600717885853
9947
0.19332144116470198 0.03723356597634589 0.21254429085419663
9948
0.19336482594448443 0.03727289641479766 0.21260668171273725
9949
0.1933453942465702 0.03726915077150404 0.2125853163543097
9950
0.1933403210701846 0.03726540588095224 0.21257830990600815
9951
0.19336555005876838 0.03728677979777321

In [26]:
dct = Article.vocab.stoi
sample = test[105].article
print(sample)
print(test[105].title)
sample = [dct['<bos>']] + [dct[ele] for ele in sample] + [dct['<eos>']]

test_sample = {}
test_sample["src"] = torch.tensor(sample, dtype=torch.long, device=device).reshape(-1, 1)
test_sample["src_len"] = [len(sample)]

print(summary(model, test_sample, Article.vocab.itos), end="\n\n")

['with', 'the', 'giants', "'", '#', '#', '#', '#', 'opener', 'at', 'houston', 'just', 'a', 'week', 'away', 'and', 'talk', 'of', 'a', 'settlement', 'of', 'the', 'major', 'league', 'baseball', 'strike', 'in', 'the', 'air', ',', 'barry', 'bonds', 'is', 'still', 'stuck', 'on', 'the', 'witness', 'stand', ',', 'fielding', 'money', 'questions', 'from', 'his', 'ex-wife', "'s", 'attorney', 'instead', 'of', 'sharpening', 'his', 'stroke', 'in', 'the', 'batting', 'cage', '.']
['with', 'giants', 'opener', 'a', 'week', 'away', 'bonds', 'is', 'on', 'the', 'witness', 'stand']


D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
D:\Program Files\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


<bos> giants giants deal on <eos>

